In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

def preprocess_images(images):
    preprocessed_images = []
    for img in images:
        img = cv2.resize(img, (299, 299))
        img = image.img_to_array(img)
        img = preprocess_input(img)
        preprocessed_images.append(img)
    return np.array(preprocessed_images)

def recognize_faces():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        resized_frame = cv2.resize(frame, (299, 299))
        img_array = image.img_to_array(resized_frame)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        frame_vector = model.predict(img_array)

        min_distance = float('inf')
        recognized_person = None
        for person, vector in feature_vectors.items():
            distance = np.linalg.norm(vector - frame_vector)
            if distance < min_distance:
                min_distance = distance
                recognized_person = person

        cv2.putText(frame, recognized_person, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow('Face Recognition', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

In [3]:
base_model = InceptionV3(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
model = Model(inputs=base_model.input, outputs=x)

In [4]:
dataset_folder = "dataset"
persons = os.listdir(dataset_folder)

feature_vectors = {}
for person in persons:
    images_folder = os.path.join(dataset_folder, person)
    images = load_images_from_folder(images_folder)
    preprocessed_images = preprocess_images(images)
    
    vectors = model.predict(preprocessed_images)
    avg_vector = np.mean(vectors, axis=0)
    
    feature_vectors[person] = avg_vector

4/4 [==============================] - 15s 4s/step


In [5]:
recognize_faces()

1/1 [==============================] - 0s 122ms/step
